In [ ]:
%reload_ext autoreload
%autoreload 2
import copy
import os
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt

from ms_stim_analysis.AnalysisTables.ms_dio_event import DioEvents, ProcessedDioEvents
from ms_stim_analysis.AnalysisTables.ms_task_identification import TaskIdentification
from ms_stim_analysis.AnalysisTables.ms_interval import EpochIntervalListName
from ms_stim_analysis.AnalysisTables.Utils.df_helpers import df_pop, df_from_data_list
from ms_stim_analysis.AnalysisTables.Utils.plot_helpers import format_ax
from ms_stim_analysis.AnalysisTables.Utils.stats_helpers import (
    average_confidence_interval,
)
from ms_stim_analysis.AnalysisTables.ms_task_performance import *
from ms_stim_analysis.Analysis.EM_fitting.AS_EM_module import EM_main
from ms_stim_analysis.AnalysisTables.ms_opto_stim_protocol import OptoStimProtocol
from ms_stim_analysis.Style.style_guide import animal_style

## Populate behavior analysis tables

In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Populate tables for behavioral analysis of W track sessions
# TODO: get list of nwbf names used in MS analysis
nwb_file_names = ["Frodo20230815_.nwb"]

for nwb_file_name in nwb_file_names:
    print(f"on {nwb_file_name}...")
    key = {"nwb_file_name": nwb_file_name}
    TaskIdentification.populate(key)
    epochs = []
    for contingency in [
        "lineartrack",
        "Lineartrack",
        "wtrack",
        "Wtrack",
        "w-track",
        "W-Track",
    ]:
        epochs.extend(
            (TaskIdentification & {**key, **{"contingency": contingency}}).fetch(
                "epoch"
            )
        )
    # epochs = np.append((TaskIdentification & {**key, **{"contingency": "wtrack"}}).fetch("epoch"),(TaskIdentification & {**key, **{"contingency": "lineartrack"}}).fetch("epoch"))
    for epoch in epochs:
        print(f"on epoch {epoch}...")
        key.update({"epoch": epoch})
        if not EpochIntervalListName & key:
            print("populating epochIntervalListName...")
            EpochIntervalListName.populate(key)
        if not DioEvents & key:
            print("populating DioEvents...")
            DioEvents.populate(key)
        if not ProcessedDioEvents & key:
            print("populating ProcessedDioEvents...")
            ProcessedDioEvents.populate(key)
        if not AlternationTaskPerformance & key:
            print("populating AlternationTaskPerformance...")
            AlternationTaskPerformanceSel().insert1(key, skip_duplicates=True)
            AlternationTaskPerformance.populate(key)

In [ ]:
x = (ProcessedDioEvents & key).fetch("KEY")[0]
(TaskIdentification & x).fetch1("contingency") in [
    "wtrack",
    "W-Track",
    "W-track",
    "wtrack",
    "w-track",
]
# ProcessedDioEvents & key
AlternationTaskPerformanceSel.insert1(x, skip_duplicates=True)

AlternationTaskPerformanceSel & x  # & key

AlternationTaskPerformance.populate(x)
# key